***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-4927  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install gensim

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import threading
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math
import gzip
import csv
import io

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  9 17:39 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ruby_bucket_327064358' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name.startswith("multi"):
        paths.append(full_path+b.name)

In [8]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [9]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [10]:
from inverted_index_gcp import InvertedIndex, MultiFileWriter, MultiFileReader

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [11]:
parquetFile = spark.read.parquet(*paths)


In [12]:
#doc_text_pairs = parquetFile.select("text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [13]:
# Count number of wiki pages
# parquetFile.count()

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [14]:
stemmer = PorterStemmer()
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id, stem = False):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  if stem is True:
      tokens = [stemmer.stem(token) for token in tokens]
  # YOUR CODE HERE
  dic = {}
  for token in tokens:
    if token not in all_stopwords:
      if token not in dic:
        dic[token] = 1
      else:
        dic[token] += 1

  def func(x):
    return (x, (id,dic[x]))

  return list(map(func, dic.keys()))

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  result = defaultdict(int)

  # Iterate over the list and accumulate the second values
  for key, value in unsorted_pl:
      result[key] += value

  # Convert the defaultdict back to a list of tuples
  result_list = list(result.items())

  return sorted(result_list, key = lambda x: x[0])


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  return postings.map(lambda x: (x[0], len(x[1])))

def partition_postings_and_write(base_dir, postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  buckets = postings.groupBy(lambda x: token2bucket_id(x[0]))
  obj = InvertedIndex()
  ret = buckets.map(lambda x: obj.write_a_posting_list(x, base_dir, bucket_name = 'ruby_bucket_327064358' ))
  return ret


# building an inverted index for the titles, with stemming

In [100]:
doc_title_pairs = parquetFile.select("title", "id").rdd

In [101]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1], True))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(base_dir = 'title_index', postings = postings_filtered).collect()
index_const_time = time() - t_start

In [103]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title_index'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [104]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/title_index/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
- [1 files][ 62.6 MiB/ 62.6 MiB]                                                
Operation completed over 1 objects/62.6 MiB.                                     


In [105]:
!gsutil ls -lh $index_dst

  62.6 MiB  2024-03-05T14:07:04Z  gs://ruby_bucket_327064358/title_index/title_index.pkl
TOTAL: 1 objects, 65640595 bytes (62.6 MiB)


# building an inverted index for the titles, without stemming

In [71]:
doc_title_pairs = parquetFile.select("title", "id").rdd

In [72]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(base_dir = 'title_index_no_stem', postings = postings_filtered).collect()
index_const_time = time() - t_start

In [73]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title_index_no_stem'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [74]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'title_index_no_stem')
# upload to gs
index_src = "title_index_no_stem.pkl"
index_dst = f'gs://{bucket_name}/title_index_no_stem/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://title_index_no_stem.pkl [Content-Type=application/octet-stream]...
- [1 files][ 67.6 MiB/ 67.6 MiB]                                                
Operation completed over 1 objects/67.6 MiB.                                     


In [75]:
!gsutil ls -lh $index_dst

  67.6 MiB  2024-03-05T15:37:46Z  gs://ruby_bucket_327064358/title_index_no_stem/title_index_no_stem.pkl
TOTAL: 1 objects, 70887731 bytes (67.6 MiB)


# building an inverted index for the body text, with stemming

In [16]:
doc_text_pairs = parquetFile.select("text", "id").rdd


In [17]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1], True))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(base_dir = 'body_index_with_stem', postings = postings_filtered).collect()
index_const_time = time() - t_start


In [18]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='body_index_with_stem'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [19]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'body_index_with_stem')
# upload to gs
index_src = "body_index_with_stem.pkl"
index_dst = f'gs://{bucket_name}/body_index_with_stem/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://body_index_with_stem.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 15.9 MiB/ 15.9 MiB]                                                
Operation completed over 1 objects/15.9 MiB.                                     


In [20]:
!gsutil ls -lh $index_dst

 15.94 MiB  2024-03-06T03:29:02Z  gs://ruby_bucket_327064358/body_index_with_stem/body_index_with_stem.pkl
TOTAL: 1 objects, 16715670 bytes (15.94 MiB)


# PageRank

import the pagerank matrix from the bucket

In [13]:

# bucket_name = 'ruby_bucket_327064358'
# storage_client = storage.Client()
# bucket = storage_client.bucket(bucket_name)
# file_path = 'pr/part-00000-0412bc70-0308-4be4-9402-8e8e513b3def-c000.csv.gz'
# blob = bucket.blob(file_path)
# blob_data = blob.download_as_string()

# # Decompress the gzip data
# decompressed_data = gzip.decompress(blob_data)

# # Decode the decompressed data as text
# decompressed_text = decompressed_data.decode('utf-8')

# # Create a StringIO object to treat the text as a file-like object
# csv_data = io.StringIO(decompressed_text)

# # Read the CSV data
# csv_reader = csv.reader(csv_data)

# page_rank_rdd = sc.parallelize(list(csv_reader))

# page_rank_rdd = page_rank_rdd.map(lambda x: (int(x[0]), float(x[1])))
                                  

# # Convert RDD to dictionary (map)
# page_rank_map = page_rank_rdd.collectAsMap()


24/03/05 11:26:29 WARN TaskSetManager: Stage 2 contains a task of very large size (110953 KiB). The maximum recommended task size is 1000 KiB.


In [14]:
# blob = bucket.blob('global_dics/page_rank.pkl')
# blob.upload_from_string(pickle.dumps(page_rank_map))

## word2vec training

In [17]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from tqdm import tqdm
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
titles = parquetFile.select("title").rdd
titles = titles.map(lambda x: x.title)
titles = titles.collect()


In [15]:
titles[:10]

['Foster Air Force Base',
 'Torino Palavela',
 'Mad About the Boy',
 'Shayne Breuer',
 'Parantaka I',
 'Arundel (UK Parliament constituency)',
 'Andrew Martinez',
 'Vancouver VooDoo',
 'Invisible plane',
 'Shopping channel']

In [24]:
from gensim.models import KeyedVectors
import gensim.downloader as api
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [25]:

# Initialize a GCS client
client = storage.Client()

# Define the GCS bucket and file path
bucket_name = "ruby_bucket_327064358"
file_name = "glove.6B.100d.txt"

# Define the local file path where you want to download the file
local_glove_file_path = "local_glove_model.txt"
local_word2vec_file_path = "local_word2vec_model.txt"

# Download the GloVe model file from GCS bucket
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name)
blob.download_to_filename(local_glove_file_path)

# Convert the GloVe model file to Word2Vec format
glove_file = local_glove_file_path
tmp_file = local_word2vec_file_path
_ = glove2word2vec(glove_file, tmp_file)

# Load the model using the Gensim API's load function
model = KeyedVectors.load_word2vec_format(tmp_file)

/tmp/ipykernel_9941/2240112156.py:20: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  _ = glove2word2vec(glove_file, tmp_file)


In [33]:
closest = model.most_similar(positive=['author'], topn=10)
closest = list(map(lambda x: x[0], closest))

closest[:2]

['writer', 'novelist']